In [8]:
%%capture
!pip install openpyxl

In [13]:
import pandas as pd
import openpyxl
from rdflib import Namespace, Graph, URIRef, Literal, BNode
from rdflib.namespace import DCTERMS, DC, XSD, RDFS, DCAT, RDF, FOAF
import os
import urllib.parse
from datetime import datetime

g = Graph()
WD = Namespace("http://www.wikidata.org/entity/")
WDT = Namespace("http://www.wikidata.org/prop/direct/")

In [14]:
fileURI = URIRef("https://g-c8521c.81de.36fe.data.globus.org/Bayraktar/dt_Bayraktar_ClinicalMetadata_mock_2022.07.21.xlsx")

g.add((fileURI, RDF.type, DCAT.Dataset))
g.add((fileURI, DCTERMS.format, WD.Q63082925))
filename = 'dt_Bayraktar_ClinicalMetadata_mock_2022.07.21.xlsx'
g.add((fileURI, RDFS.label, Literal(filename, datatype=XSD.string)))
datecreated = os.path.getctime(filename)
g.add((fileURI, DCTERMS.issued, Literal(datetime.fromtimestamp(datecreated).strftime('%Y-%m-%dT%H:%M:%S'), datatype=XSD.date)))
datemodified = os.path.getmtime(filename)
g.add((fileURI, DCTERMS.modified, Literal(datetime.fromtimestamp(datemodified).strftime('%Y-%m-%dT%H:%M:%S'), datatype=XSD.date)))

<Graph identifier=Nc26dc543f80b4b00aecdb6329e712d1d (<class 'rdflib.graph.Graph'>)>

In [15]:
print(g.serialize(format="turtle"))

@prefix dcat: <http://www.w3.org/ns/dcat#> .
@prefix dcterms: <http://purl.org/dc/terms/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<https://g-c8521c.81de.36fe.data.globus.org/Bayraktar/dt_Bayraktar_ClinicalMetadata_mock_2022.07.21.xlsx> a dcat:Dataset ;
    rdfs:label "dt_Bayraktar_ClinicalMetadata_mock_2022.07.21.xlsx"^^xsd:string ;
    dcterms:format <http://www.wikidata.org/entity/Q63082925> ;
    dcterms:issued "2022-08-23"^^xsd:date ;
    dcterms:modified "2022-08-22"^^xsd:date .




In [16]:
pd.set_option('display.max_columns', None)
xls = pd.ExcelFile(filename)
subjects = pd.read_excel(xls)
subjects

,Sample ID,MPBTP (Sequencing Data),# of blocks,Sex,Age (optional),Tumour Spatial Information,Pathology - pre-surgical,Histopathological Assignment,Histopathology - Microscopic Description,Mutations,Methylation Status (>10% = METHYLATED),Other Variants,Covid test,Other comments,Spatial Location
0,Patient A,NaN,5,M,54,Right Temporal Lobe,GBM,"Glioblastoma, IDH-wiltype (CNS WHO Grade 4)",Cellular glial neoplasm with elongated nuclei ...,IDH-WT (IHC and Sequencing)\nATRX - Retained,MGMT Promotor Methylation - 2% - ABSENT,EGFR Amplification;\nCDKN2A Deletion;\nPTEN De...,Negative,A1 has a duplicate block,Wasn’t able to get stealth location informatio...
1,Patient B,NaN,4,F,82,Right Temporal Lobe,GBM,"Glioblastoma, IDH-wiltype (CNS WHO Grade 4)",Pleomorphic astrocytes with numerous gemistocy...,IDH-WT (IHC and Sequencing)\nATRX - Retained,MGMT Promotor Methylation - 5% - ABSENT,CDKN2A Deletion; \nPTEN Deletion,Negative,-,"Anterior, Lateral, Posterior and Medial sample..."
2,Patient C,NaN,4,F,41,Right Frontal Lobe,GBM,"Oligodendroglioma, IDH-mut and 1p/19q co-delet...",Diffusely infiltrative glial neoplasm with mod...,IDH1-mut R132H (IHC and Sequencing)\nATRX - Re...,No tested,1p/19q Co-deletion; \nTP53 P152L,Negative,Sampled by Richard,"Superior, Anterior, Deep and Inferior samples ..."


In [6]:
BASE = Namespace("http://idr.semscape.org/")
SUBJECT = Namespace(BASE.subject)
OBO = Namespace("http://purl.obolibrary.org/obo/")
g.bind("obo", "http://purl.obolibrary.org/obo/")
SCHEMA = Namespace("https://schema.org/")
SIO = Namespace("http://semanticscience.org/resource/")
NODE = Namespace("http://idr.semscape.org/nodeURI/")

In [7]:
for index, row in subjects.iterrows():
    # Row Subject
    patientID = rllib.parse.quote(row["Sample ID"])
    g.add((SUBJECT["/"+patientID], RDF.type, OBO.NCIT_C16960))
    g.add((SUBJECT["/"+patientID], DCTERMS.identifier, Literal(patientID, datatype=XSD.string)))
    
    subjectProperties = BNode()
    g.add((SUBJECT["/"+patientID], OBO.RO_0000053, subjectProperties))
    # Row Age
    ## please store birthdate, since Age does not age well in data. 
    
    # g.add((subjectProperties, SCHEMA.birthDate, Literal(str(int(row["Age (year of birth)"])), datatype=XSD.date)))
    
      

    
    # Family history
    familyhistory = BNode()
    g.add((familyhistory, RDF.type, RDF.Bag))
    g.add((familyhistory, RDFS.domain, OBO.NCIT_C17726))
    g.add((subjectProperties, OBO.BFO_0000185, familyhistory))
    if row["Premen BRCA"] == 1:
        premenbrca = BNode()
        g.add((familyhistory, RDFS.member, premenbrca))
        g.add((premenbrca, RDF.type, OBO.NCIT_C49152))
        g.add((premenbrca, RDFS.domain, OBO.DOID_1612))
        g.add((premenbrca, URIRef("http://rs.tdwg.org/dwc/terms/lifeStage"), OBO.OMIT_0018003))
        g.add((premenbrca, RDF.value, Literal(int(row["Premen BRCA"]), datatype=XSD.int)))

    if row["Post BRCA"] == 1:
        postbrca = BNode()
        g.add((familyhistory, RDFS.member, postbrca))
        g.add((postbrca, RDF.type, OBO.NCIT_C49152))
        g.add((postbrca, RDFS.domain, OBO.DOID_1612))
        g.add((postbrca, URIRef("http://rs.tdwg.org/dwc/terms/lifeStage"), OBO.OMIT_0023947))
        g.add((postbrca, RDF.value, Literal(int(row["Post BRCA"]), datatype=XSD.int)))

    if row["OVCA"] == 1:
        ovca = BNode()
        g.add((familyhistory, RDFS.member, ovca))
        g.add((ovca, RDF.type, OBO.NCIT_C49152))
        g.add((ovca, RDFS.domain, OBO.DOID_2394))
        g.add((ovca, RDF.value, Literal(int(row["OVCA"]), datatype=XSD.int)))
        
    if row["Male BrCA"] == 1:
        malebrca = BNode()
        g.add((familyhistory, RDFS.member, malebrca))
        g.add((malebrca, RDF.type, OBO.NCIT_C15709))
        g.add((malebrca, RDFS.domain, OBO.DOID_1614))
        g.add((malebrca, RDF.value, Literal(int(row["OVCA"]), datatype=XSD.int)))
        
    
    if row["BRCA1.1"] == 1:
        fambrca1test = BNode()
        g.add((familyhistory, RDFS.member, fambrca1test))
        g.add((fambrca1test, RDF.type, OBO.NCIT_C15709))
        g.add((fambrca1test, RDFS.domain, OBO.NCIT_C17965))
         
    if row["BRCA2.1"] == 1:
        fambrca2test = BNode()
        g.add((familyhistory, RDFS.member, fambrca2test))
        g.add((fambrca1test, RDF.type, OBO.NCIT_C15709))
        g.add((fambrca2test, RDFS.domain, OBO.NCIT_C18120))
    if row["PALB2"] == 1:
        fampalb2test = BNode()
        g.add((familyhistory, RDFS.member, fampalb2test))
        g.add((fambrca1test, RDF.type, OBO.NCIT_C15709))
        g.add((fampalb2test, RDFS.domain, OBO.NCIT_C84945))
    if row["Other"] == 1:
        othertest = BNode()
        g.add((familyhistory, RDFS.member, othertest))
        g.add((othertest, RDF.type, OBO.NCIT_C15709))
        g.add((fampalb2test, RDFS.domain, OBO.NCIT_C113067))
   
    # Prior Diagnosis
    medicalhistory = BNode()
    g.add((medicalhistory, RDF.type, RDF.Bag))
    g.add((medicalhistory, RDFS.domain, OBO.SIO_010673))
    g.add((subjectProperties, OBO.BFO_0000185, medicalhistory))
    if row["Breast.1"] == 1:
        past_diagnosis = BNode()
        g.add((medicalhistory, RDFS.member, past_diagnosis))
        g.add((past_diagnosis, RDF.type, OBO.C4872))
        g.add((past_diagnosis, DC.date, Literal(str(int(row["Year  Dx"]))+"-"+str(int(row["Month Dx"])), datatype=XSD.date)))
      
    # Menopause
    ## premenopause
    if row["Pre"] == 1:
        g.add((subjectProperties, URIRef("http://rs.tdwg.org/dwc/terms/lifeStage"), OBO.OMIT_0018003))
    if row["Peri"] == 1:
        g.add((subjectProperties, URIRef("http://rs.tdwg.org/dwc/terms/lifeStage"), OBO.OMIT_0023947))
    if row["Post"] == 1:
        g.add((subjectProperties, URIRef("http://rs.tdwg.org/dwc/terms/lifeStage"), OBO.OMIT_0018004))

    subjectArchivedSpecimen = BNode()
    g.add((SUBJECT["/"+row["Subject"]], OBO.OBI_0100051, subjectArchivedSpecimen))
    g.add((subjectArchivedSpecimen, RDF.type, OBO.NCIT_C19157))
    
    #Diagnosis
    diagnosis = BNode()
    g.add((subjectArchivedSpecimen, OBO.RO_0040035, diagnosis))
    g.add((medicalhistory, RDFS.member, diagnosis))
    if row["Invasive CA"] == 1:
        g.add((diagnosis, RDF.type, OBO.NCIT_C9480)) # Invasive carcinoma
    if row["DCIS"] == 1:
        g.add((diagnosis, RDF.type, OBO.NCIT_C2924)) # DCIS
    if row["LCIS"] == 1:
        g.add((diagnosis, RDF.type, OBO.NCIT_C137839)) # LCIS = Breast Pleomorphic Lobular Carcinoma In Situ?
    if row["Atypia"] == 1:
        g.add((diagnosis, RDF.type, OBO.NCIT_C9478)) # Atypia
    g.add((diagnosis, DC.date, Literal(str(int(row["Year"]))+"-"+str(int(row["Month"])), datatype=XSD.date)))
 
    # How should "Normal" be interpreted?
    
    tfinding = BNode()
    g.add((diagnosis, OBO.NCIT_R108, tfinding))
    g.add((tfinding, RDF.type, OBO.NCIT_C48879))
    g.add((tfinding, RDF.value, Literal("T"+str(row["T"]), datatype=XSD.string)))
    
    nfinding = BNode()
    g.add((diagnosis, OBO.NCIT_R108, nfinding))
    g.add((nfinding, RDF.type, OBO.NCIT_C48879))
    g.add((nfinding, RDF.value, Literal("N"+str(row["N"]), datatype=XSD.string)))
    
    mfinding = BNode()
    g.add((diagnosis,OBO.NCIT_R108, mfinding))
    g.add((mfinding, RDF.type, OBO.NCIT_C48879))
    g.add((mfinding, RDF.value, Literal("M"+str(row["M"]), datatype=XSD.string)))
    
    # Expression
    if not pd.isna(row["ER"]):
        er = BNode()
        g.add((subjectArchivedSpecimen, OBO.TXPO_0001873, er))
        g.add((er, RDF.type, OBO.NCIT_C25209))
        g.add((er, RDFS.range, OBO.NCIT_C16150))
        if row["ER"] == 1:
            g.add((er, RDF.value, Literal(True)))
        if row["ER"] == 0:
            g.add((er, RDF.value, Literal(False)))
            
    if not pd.isna(row["PR"]):
        pr = BNode()
        g.add((subjectArchivedSpecimen, OBO.TXPO_0001873, pr))
        g.add((pr, RDF.type, OBO.NCIT_C25209))
        g.add((pr, RDFS.range, OBO.NCIT_C16149))
        if row["PR"] == 1:
            g.add((pr, RDF.value, Literal(True)))
        if row["PR"] == 0:
            g.add((pr, RDF.value, Literal(False)))
            
    if not pd.isna(row["HER2"]):
        her = BNode()
        g.add((subjectArchivedSpecimen, OBO.TXPO_0001873, her))
        g.add((her, RDF.type, OBO.NCIT_C25209))
        g.add((her, RDFS.range, OBO.NCIT_C184942))
        if row["HER2"] == 1:
            g.add((her, RDF.value, Literal(True)))
        if row["HER2"] == 0:
            g.add((her, RDF.value, Literal(False)))
    
    if not pd.isna(row["Ki67 (%)"]):
        ki67 = BNode()
        g.add((subjectArchivedSpecimen, OBO.TXPO_0001873, ki67))
        g.add((her, RDF.type, OBO.NCIT_C25209))
        g.add((her, RDF.value, Literal(row["Ki67 (%)"])))
        g.add((her, SIO.SIO_000221, SIO.SIO_001413))
        
            
    # collection method
    collectionmethod = BNode()
    g.add((subjectArchivedSpecimen, OBO.HSO_0000288, collectionmethod))
    g.add((collectionmethod, RDF.type, OBO.NCIT_C70700))
    if row["core Biopsy (y/n)"] == 1:
        g.add((collectionmethod, RDF.value, OBO.NCIT_C15189))
    if row["Excisional biopsy"] == 1:
        g.add((collectionmethod, RDF.value, OBO.NCIT_C51633))
    if row["Mastectomy"] == 1:
        g.add((collectionmethod, RDF.value, OBO.NCIT_C15279))
    
    # Hypoxia time (mins)
    specimenHandling = BNode()
    g.add((subjectArchivedSpecimen, OBO.RO_0002334, specimenHandling))
    g.add((specimenHandling, RDF.type, OBO.NCIT_C179745))
    g.add((specimenHandling, RDFS.range, OBO.OMIT_0016357))
    g.add((specimenHandling, RDF.value, Literal(row["Hypoxia time (mins)"])))
    g.add((specimenHandling, SIO.SIO_000221, SIO.SIO_000434)) # unit minutes 
    
    # Formalin Fix Time (mins)
    specimenHandling = BNode()
    g.add((subjectArchivedSpecimen, OBO.RO_0002334, specimenHandling))
    g.add((specimenHandling, RDF.type, OBO.NCIT_C179745))
    g.add((specimenHandling, RDFS.range, OBO.OBIB_0000718))
    g.add((specimenHandling, RDF.value, Literal(row["Formalin Fix Time"])))
    g.add((specimenHandling, SIO.SIO_000221, SIO.SIO_000434)) # unit minutes 
    
    # Mutations
    if row["BRCA1"] == 1:
        known_mutation = BNode()
        g.add((SUBJECT["/"+row["Subject"]], OBO.CLO_0037333, known_mutation))
        g.add((known_mutation, RDF.type, OBO.NCIT_C36391))
        g.add((known_mutation, RDFS.range, OBO.NCIT_C17965))
        g.add((known_mutation, RDFS.comment, Literal(row["Specific MT (text)"])))
    if row["BRCA2"] == 1:
        known_mutation = BNode()
        g.add((SUBJECT["/"+row["Subject"]], OBO.CLO_0037333, known_mutation))
        g.add((known_mutation, RDF.type, OBO.NCIT_C36391))
        g.add((known_mutation, RDFS.range, OBO.NCIT_C18120))
        g.add((known_mutation, RDFS.comment, Literal(row["Specific MT (text)"])))
    if row["other"] == 1:
        known_mutation = BNode()
        g.add((SUBJECT["/"+row["Subject"]], OBO.CLO_0037333, known_mutation))
        g.add((known_mutation, RDF.type, OBO.NCIT_C36391))
        g.add((known_mutation, RDFS.range, OBO.NCIT_C16612))
        g.add((known_mutation, RDFS.comment, Literal(row["Specific MT (text)"])))
    
    # Treatment
    if row["NACT (y/n)"] == 1:
        treatment = BNode()
        g.add((subjectArchivedSpecimen, WDT.P2176, treatment))
        g.add((treatment, RDF.type, OBO.NCIT_C15665))
        if not pd.isna(row["Year.1"]):
            g.add((treatment, SCHEMA.startDate, Literal(str(row["Year.1"])+"-"+str(row["Month.1"]), datatype=XSD.date)))
        if not pd.isna(row["Year.2"]):
            g.add((treatment, SCHEMA.endDate, Literal(str(row["Year.2"])+"-"+str(row["Month.2"]), datatype=XSD.date)))
    if row["PARP-I y/n"] == 1:
        treatment = BNode()
        g.add((subjectArchivedSpecimen, WDT.P2176, treatment))
        g.add((treatment, RDF.type, OBO.NCIT_C62554))
        if not pd.isna(row["Year.1"]):
            g.add((treatment, SCHEMA.startDate, Literal(str(row["Year.1"])+"-"+str(row["Month.1"]), datatype=XSD.date)))
        if not pd.isna(row["Year.2"]):
            g.add((treatment, SCHEMA.endDate, Literal(str(row["Year.2"])+"-"+str(row["Month.2"]), datatype=XSD.date)))
   
    
        
        
    
    
    
    

KeyError: 'Subject'

In [ ]:
print(g.serialize(destination=filename+".ttl", format="turtle"))
print(g.serialize(format="turtle"))


In [ ]:
for node in g.all_nodes():
    if isinstance(node, URIRef):
        try:
            g.parse(node)
        except:
            print(node)

In [ ]:
g.parse("http://www.wikidata.org/entity/Q63082925.ttl")

# Extract schema

In [ ]:
from shexer.shaper import Shaper
from shexer.consts import NT, TURTLE

q = "select ?class where { ?item rdf:type ?class }"
target_classes = []
x = g.query(q)
for target_class in x:
  if str(target_class["class"]) not in target_classes:
    target_classes.append(str(target_class["class"]))

shex_target_file = filename+".shex"

shaper = Shaper(target_classes=target_classes,
                rdflib_graph=g,
                input_format=TURTLE,
                )  # Default rdf:type
            
shaper.shex_graph(output_file=shex_target_file)


In [ ]:
command = f"cat {filename}.shex"

In [ ]:
!command

In [ ]:
import os
os.system(command)